# Modeling for Snow Depth Prediction

This notebook is focused on preparing the data for modeling, addressing multicollinearity, and creating training, validation, and testing sets. We will proceed with the following steps:
- Loading the Processed Data
- Creating Lag Features and Derived Variables
- Handling Multicollinearity
- Splitting the Data into Training, Validation, and Testing Sets

### 1. Loading Libraries and Processed Data

In [3]:
import pandas as pd
import numpy as np
import os
import sys
import logging

# Determine the project root directory (assuming the notebook is in the 'notebooks' folder)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to sys.path")

# Verify the update
print("Current sys.path:")
for path in sys.path:
    print(path)

Added /workspace/SkiSnow to sys.path
Current sys.path:
/workspace/SkiSnow
/home/gitpod/.pyenv/versions/3.12.6/lib/python312.zip
/home/gitpod/.pyenv/versions/3.12.6/lib/python3.12
/home/gitpod/.pyenv/versions/3.12.6/lib/python3.12/lib-dynload

/workspace/SkiSnow/venv/lib/python3.12/site-packages


# Import modeling modules

In [4]:
from src.models.model_training import train_random_forest, evaluate_model
from src.models.model_evaluation import plot_actual_vs_predicted, plot_feature_importances
from src.models.utils import save_model, save_split_datasets

# Configure logging

In [5]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

### 2. Load Processed Data

In [6]:
# Adjust the path based on your current working directory
data_path = os.path.join('..', 'data', 'processed', 'processed_data_for_modeling.csv')

# Load the combined processed data
model_data = pd.read_csv(data_path)

# Ensure 'date' column is in datetime format
model_data['date'] = pd.to_datetime(model_data['date'])

# Display the shape to confirm loading
logging.info(f"Loaded combined data with shape {model_data.shape}")

2024-10-28 14:37:51,306 - INFO - Loaded combined data with shape (75592, 12)


### 3. Encoding Categorical Variables

In [7]:
# One-Hot Encode the 'resort' feature
model_data = pd.get_dummies(model_data, columns=['resort'], drop_first=True)

# Display the columns to confirm encoding
logging.info(f"Columns after encoding: {model_data.columns.tolist()}")

2024-10-28 14:38:02,206 - INFO - Columns after encoding: ['date', 'temperature_min', 'temperature_max', 'precipitation_sum', 'snow_depth', 'season_id', 'is_operating_season', 'snow_depth_lag1', 'snow_depth_lag7', 'temperature_avg', 'temperature_avg_squared', 'resort_kitzbuhel', 'resort_kranjska_gora', 'resort_krvavec', 'resort_mariborsko_pohorje', 'resort_sestriere', 'resort_solden', 'resort_st_anton', 'resort_st_moritz', 'resort_val_gardena']


### 4. Preparing Features and Target Variable

In [8]:
# Filter data for operating season
data_operating = model_data[model_data['is_operating_season'] == True].reset_index(drop=True)
logging.info(f"Data shape after filtering for operating season: {data_operating.shape}")

# Define the target variable
y = data_operating['snow_depth']

# Exclude 'date' and 'snow_depth' from features
feature_columns = [col for col in data_operating.columns if col != 'date']

# Create the features DataFrame
X = data_operating[feature_columns].copy()

logging.info(f"Initial shape of X (including 'snow_depth'): {X.shape}")
logging.info(f"Initial shape of y: {y.shape}")

2024-10-28 14:38:08,007 - INFO - Data shape after filtering for operating season: (36461, 20)
2024-10-28 14:38:08,011 - INFO - Initial shape of X (including 'snow_depth'): (36461, 19)
2024-10-28 14:38:08,012 - INFO - Initial shape of y: (36461,)


### 5. Check for Non-Numeric Columns & Handle "season_id"

In [9]:
# Check for non-numeric columns
non_numeric_cols = X.select_dtypes(include=['object', 'bool']).columns.tolist()
logging.info(f"Non-numeric columns before processing: {non_numeric_cols}")

# Handle 'season_id' column
# One-Hot Encode 'season_id' if it's in the features
if 'season_id' in X.columns:
    # One-Hot Encode 'season_id'
    season_dummies = pd.get_dummies(X['season_id'], prefix='season', drop_first=True)
    # Drop 'season_id' and add the encoded variables
    X = pd.concat([X.drop(columns=['season_id']), season_dummies], axis=1)
    logging.info("One-Hot Encoded 'season_id' and updated X.")
else:
    logging.warning("'season_id' column not found in features.")

# Convert boolean columns to integers
bool_columns = X.select_dtypes(include=['bool']).columns.tolist()
if bool_columns:
    X[bool_columns] = X[bool_columns].astype(int)
    logging.info(f"Converted boolean columns to integers: {bool_columns}")

# Verify all columns are now numeric
non_numeric_cols = X.select_dtypes(include=['object']).columns.tolist()
logging.info(f"Non-numeric columns after processing: {non_numeric_cols}")
logging.info("Data types after processing:")
logging.info(X.dtypes)

if not non_numeric_cols:
    logging.info("All features are now numeric and ready for Polynomial Features.")
else:
    logging.error("There are still non-numeric features that need to be encoded.")

2024-10-28 14:38:20,029 - INFO - Non-numeric columns before processing: ['season_id', 'is_operating_season', 'resort_kitzbuhel', 'resort_kranjska_gora', 'resort_krvavec', 'resort_mariborsko_pohorje', 'resort_sestriere', 'resort_solden', 'resort_st_anton', 'resort_st_moritz', 'resort_val_gardena']
2024-10-28 14:38:20,040 - INFO - One-Hot Encoded 'season_id' and updated X.
2024-10-28 14:38:20,079 - INFO - Converted boolean columns to integers: ['is_operating_season', 'resort_kitzbuhel', 'resort_kranjska_gora', 'resort_krvavec', 'resort_mariborsko_pohorje', 'resort_sestriere', 'resort_solden', 'resort_st_anton', 'resort_st_moritz', 'resort_val_gardena', 'season_1991-1992', 'season_1992-1993', 'season_1993-1994', 'season_1994-1995', 'season_1995-1996', 'season_1996-1997', 'season_1997-1998', 'season_1998-1999', 'season_1999-2000', 'season_2000-2001', 'season_2001-2002', 'season_2002-2003', 'season_2003-2004', 'season_2004-2005', 'season_2005-2006', 'season_2006-2007', 'season_2007-2008', '

### 6. Lag Features

Incorporate lagged values of snow depth and other relevant features to capture temporal dependencies.

In [10]:
# Define lag periods
lags = [1, 7, 14, 21]  # You can adjust these based on domain knowledge

# Features to create lag features for
lag_features = ['snow_depth', 'temperature_avg', 'precipitation_sum']

for feature in lag_features:
    for lag in lags:
        lag_col = f"{feature}_lag{lag}"
        if feature in X.columns:
            X[lag_col] = X[feature].shift(lag)
            logging.info(f"Created lag feature '{lag_col}'.")
        else:
            logging.warning(f"Feature '{feature}' not found in X. Skipping lag creation for this feature.")

# Drop rows with NaN values resulting from lagging
# Ensure that any rows dropped are reflected in both X and y
initial_length = len(X)
X = X.dropna().reset_index(drop=True)
y = y.iloc[X.index].reset_index(drop=True)

logging.info(f"Shape of X after adding lag features: {X.shape}")
logging.info(f"Shape of y after adding lag features: {y.shape}")

2024-10-28 14:38:29,347 - INFO - Created lag feature 'snow_depth_lag1'.
2024-10-28 14:38:29,349 - INFO - Created lag feature 'snow_depth_lag7'.
2024-10-28 14:38:29,351 - INFO - Created lag feature 'snow_depth_lag14'.
2024-10-28 14:38:29,353 - INFO - Created lag feature 'snow_depth_lag21'.
2024-10-28 14:38:29,354 - INFO - Created lag feature 'temperature_avg_lag1'.
2024-10-28 14:38:29,355 - INFO - Created lag feature 'temperature_avg_lag7'.
2024-10-28 14:38:29,356 - INFO - Created lag feature 'temperature_avg_lag14'.
2024-10-28 14:38:29,357 - INFO - Created lag feature 'temperature_avg_lag21'.
2024-10-28 14:38:29,358 - INFO - Created lag feature 'precipitation_sum_lag1'.
2024-10-28 14:38:29,359 - INFO - Created lag feature 'precipitation_sum_lag7'.
2024-10-28 14:38:29,361 - INFO - Created lag feature 'precipitation_sum_lag14'.
2024-10-28 14:38:29,362 - INFO - Created lag feature 'precipitation_sum_lag21'.
2024-10-28 14:38:29,466 - INFO - Shape of X after adding lag features: (36440, 61)

### 7. Handling Multicollinearity

Before splitting the data, it's important to check for multicollinearity among features.

Calculating the Variance Inflation Factor (VIF) for each feature is important.

In [11]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
    
def calculate_vif(X):
    """
    Calculate Variance Inflation Factor (VIF) for each feature in the DataFrame.
    
    Parameters:
    - X (pd.DataFrame): Feature matrix.
    
    Returns:
    - pd.DataFrame: DataFrame containing features and their VIF values.
    """
    vif = pd.DataFrame()
    vif['feature'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif

# Exclude 'snow_depth' from VIF calculation if present
features_to_exclude = ['snow_depth']
X_vif = X.drop(columns=features_to_exclude, errors='ignore').copy()

# Calculate initial VIF
vif_data = calculate_vif(X_vif)
logging.info("Initial Variance Inflation Factors:")
logging.info(vif_data.sort_values('VIF', ascending=False))

# Identify features with VIF > 10
high_vif = vif_data[vif_data['VIF'] > 10]['feature'].tolist()
logging.info(f"Features with VIF > 10: {high_vif}")

# Iteratively remove features with high VIF
for feature in high_vif:
    if feature in X.columns:
        X = X.drop(columns=[feature])
        logging.info(f"Dropped feature '{feature}' due to high VIF.")
        # Recalculate VIF after dropping the feature
        X_vif = X.drop(columns=features_to_exclude, errors='ignore').copy()
        vif_data = calculate_vif(X_vif)
        logging.info(vif_data.sort_values('VIF', ascending=False))
    else:
        logging.warning(f"Feature '{feature}' already removed.")

# Final VIF after removal
final_vif = calculate_vif(X_vif)
logging.info("Final Variance Inflation Factors:")
logging.info(final_vif.sort_values('VIF', ascending=False))

/workspace/SkiSnow/venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
2024-10-28 14:39:25,296 - INFO - Initial Variance Inflation Factors:
2024-10-28 14:39:25,298 - INFO -                       feature        VIF
0             temperature_min        inf
1             temperature_max        inf
6             temperature_avg        inf
3         is_operating_season  89.762142
52       temperature_avg_lag1   6.315753
13              resort_solden   6.090312
8            resort_kitzbuhel   5.692266
14            resort_st_anton   5.062711
10             resort_krvavec   4.782867
9        resort_kranjska_gora   4.745074
11  resort_mariborsko_pohorje   4.697186
5             snow_depth_lag7   4.509378
50           snow_depth_lag14   4.164787
12           resort_sestriere   3.849643
4             snow_depth_lag1   3.313765
15           resort_st_moritz   3.253613
51         

### 8. Adding Polynomial Features

Now that all non-numeric columns have been handled, we can safely apply Polynomial Features to capture non-linear relationships.

In [12]:
from sklearn.preprocessing import PolynomialFeatures

# Define features to apply PolynomialFeatures
selected_features = ['temperature_avg', 'precipitation_sum']

# Check which selected features are present in X
available_features = [feature for feature in selected_features if feature in X.columns]
missing_features = [feature for feature in selected_features if feature not in X.columns]

if missing_features:
    logging.warning(f"Cannot apply PolynomialFeatures to missing features: {missing_features}")
    logging.info("Proceeding with available features only.")
else:
    logging.info("All selected features are available for PolynomialFeatures.")

if available_features:
    # Initialize PolynomialFeatures with degree 2
    poly = PolynomialFeatures(degree=2, include_bias=False)
    
    # Fit and transform only the available features
    X_poly_selected = poly.fit_transform(X[available_features])
    
    # Get the new feature names for selected features
    poly_features_selected = poly.get_feature_names_out(available_features)
    
    # Create a DataFrame for polynomial features
    X_poly_df = pd.DataFrame(X_poly_selected, columns=poly_features_selected)
    
    # Drop the original selected features from X
    X = X.drop(columns=available_features)
    
    # Concatenate the polynomial features back to X
    X = pd.concat([X.reset_index(drop=True), X_poly_df.reset_index(drop=True)], axis=1)
    
    logging.info(f"Shape of X after adding polynomial features to selected features: {X.shape}")
else:
    logging.warning("No features available for PolynomialFeatures. Skipping this step.")

2024-10-28 14:47:06,010 - WARNING - Cannot apply PolynomialFeatures to missing features: ['temperature_avg']
2024-10-28 14:47:06,012 - INFO - Proceeding with available features only.
2024-10-28 14:47:06,023 - INFO - Shape of X after adding polynomial features to selected features: (36440, 58)


### 9. Splitting the Data

Since the data is time-series data, it's important to split it in a way that respects the temporal order and thus avoid data leakage.

#### 9 (a) Split the data into training, validation, and test sets using time-based splits.

In [13]:
# Define the sizes for training, validation, and testing sets
total_length = len(X)
train_size = int(0.7 * len(X))
val_size = int(0.15 * len(X))
test_size = total_length - train_size - val_size

# Split the data
X_train = X.iloc[:train_size].reset_index(drop=True)
y_train = y.iloc[:train_size].reset_index(drop=True)

X_val = X.iloc[train_size:train_size + val_size].reset_index(drop=True)
y_val = y.iloc[train_size:train_size + val_size].reset_index(drop=True)

X_test = X.iloc[train_size + val_size:].reset_index(drop=True)
y_test = y.iloc[train_size + val_size:].reset_index(drop=True)

logging.info(f"Training set size: {X_train.shape}, y_train size: {y_train.shape}")
logging.info(f"Validation set size: {X_val.shape}, y_val size: {y_val.shape}")
logging.info(f"Test set size: {X_test.shape}, y_test size: {y_test.shape}")

# Verify alignment
assert X_train.shape[0] == y_train.shape[0], "Mismatch in training set."
assert X_val.shape[0] == y_val.shape[0], "Mismatch in validation set."
assert X_test.shape[0] == y_test.shape[0], "Mismatch in test set."

logging.info("All splits are aligned correctly.")

2024-10-28 14:47:13,965 - INFO - Training set size: (25508, 58), y_train size: (25508,)
2024-10-28 14:47:13,966 - INFO - Validation set size: (5466, 58), y_val size: (5466,)
2024-10-28 14:47:13,967 - INFO - Test set size: (5466, 58), y_test size: (5466,)
2024-10-28 14:47:13,968 - INFO - All splits are aligned correctly.


### 10. Proceeding with Modeling

The data is now prepared.  We can proceed to build and evaluate the models.

#### 10 (a) Training The Model

In [14]:
# Define parameter grid
param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

# Train the Random Forest Regressor
best_rf_model, best_params = train_random_forest(
    X_train=X_train,
    y_train=y_train,
    param_dist=param_dist,
    n_iter=10,
    cv=3,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Predict and evaluate on Validation Set
val_metrics = evaluate_model(best_rf_model, X_val, y_val, dataset_name="Validation")

# Predict on Test Set
test_metrics = evaluate_model(best_rf_model, X_test, y_test, dataset_name="Test")

2024-10-28 14:47:27,668 - INFO - Starting RandomizedSearchCV for Random Forest Regressor...


Fitting 3 folds for each of 10 candidates, totalling 30 fits


/workspace/SkiSnow/venv/lib/python3.12/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
2024-10-28 14:49:13,304 - INFO - RandomizedSearchCV completed.
2024-10-28 14:49:13,305 - INFO - Best Parameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20}
2024-10-28 14:49:13,306 - INFO - Best Score: -3581.9584471344992
2024-10-28 14:49:13,419 - INFO - 
Validation Set Evaluation:
2024-10-28 14:49:13,421 - INFO - Mean Squared Error: 1075.79
2024-10-28 14:49:13,422 - INFO - R-squared: 0.88
2024-10-28 14:49:13,543 - INFO - 
Test Set Evaluation:
2024-10-28 14:49:13,544 - INFO - Mean Squared Error: 2971.35
2024-10-28 14:49:13,545 - INFO - R-squared: 0.93


### 11. Plotting Evaluation Metrics

In [15]:
# Plot Actual vs Predicted for Validation Set
plot_actual_vs_predicted(
    y_true=y_val,
    y_pred=best_rf_model.predict(X_val),
    dataset_name="Validation",
    save_dir=os.path.join('..', 'src', 'models', 'plots')
)

# Plot Actual vs Predicted for Test Set
plot_actual_vs_predicted(
    y_true=y_test,
    y_pred=best_rf_model.predict(X_test),
    dataset_name="Test",
    save_dir=os.path.join('..', 'src', 'models', 'plots')
)

# Plot Feature Importances
plot_feature_importances(
    model=best_rf_model,
    feature_names=X_train.columns.tolist(),
    save_dir=os.path.join('..', 'src', 'models', 'plots')
)

2024-10-28 15:04:35,372 - INFO - Actual vs. Predicted plot saved to ../src/models/plots/actual_vs_predicted_validation.png
2024-10-28 15:04:38,660 - INFO - Actual vs. Predicted plot saved to ../src/models/plots/actual_vs_predicted_test.png
/workspace/SkiSnow/src/models/model_evaluation.py:58: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=sorted_importances, y=sorted_features, palette='viridis')
2024-10-28 15:04:39,826 - INFO - Feature importances plot saved to ../src/models/plots/feature_importances.png


### 11. Saving the Model

In [17]:
# Define the directory to save the model
model_save_dir = os.path.join('..', 'src', 'models')

# Define the path to save the model
model_save_path = os.path.join(model_save_dir, 'best_rf_model.pkl')

# Save the trained model
save_model(best_rf_model, model_save_path)

2024-10-28 15:06:10,304 - INFO - Model successfully saved to ../src/models/best_rf_model.pkl


### 12. Saving the Split Datasets

After training and evaluating the model, we'll save the split datasets (`X_train.csv`, `y_train.csv`, `X_val.csv`, `y_val.csv`, `X_test.csv`, `y_test.csv`) to the `data/processed/modeling_data/` directory. This ensures that these datasets can be easily accessed for future evaluations, residual analysis, and other analyses without the need to regenerate them.

In [18]:
# Define the directory to save the split datasets
modeling_data_dir = os.path.join('..', 'data', 'processed', 'modeling_data')

# Save the split datasets
save_split_datasets(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    save_dir=modeling_data_dir
)

2024-10-28 15:07:16,391 - INFO - Split datasets successfully saved to ../data/processed/modeling_data
